In [1]:
import time
import pyspark
import numpy as np
import pandas as pd
import itertools as it
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [2]:
try:
    sc.stop()
except Exception as e:
    print(e)
try:
    sessn.stop()
except Exception as e:
    print(e)
try:
    sc.SparkContext.stop()
except Exception as e:
    print(e)

name 'sessn' is not defined
'SparkContext' object has no attribute 'SparkContext'


In [3]:
def get_md(u, v, IV):
    """
    Compute the Mahalanobis distance (variance).
    The Mahalanobis distance between  `u` and `v`, is defined as
    .. math::{ (u-v) VI (u-v)^T }
    
    Parameters
    ----------
    u : (N,) array_like
        Input array.
    v : (N,) array_like
        Input array.
    VI : ndarray
        The inverse of the covariance matrix.

    Returns
    -------
    mahalanobis : double
        The Mahalanobis distance between vectors `u` and `v`.
        
    """
    u_v = u - v 
    lpart = np.dot(u_v, IV)
    fpart = np.dot(lpart, u_v.T)
    return fpart

In [4]:
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '8g'), 
                                   ('spark.executor.instances', '10'),
                                   ('spark.executor.cores', '4'), 
                                   ('spark.driver.cores','4'), 
                                   ('spark.driver.memory','8g'),
                                   ('spark.yarn.queue', 'araadh_q1.arapi_sq1'),
                                  ('spark.yarn.executor.memoryOverhead', '1g')])
sessn = SparkSession.builder.appName("HPP").config(conf=conf).enableHiveSupport().getOrCreate().newSession()
sessn

#### Reading the Data

In [5]:
m_table = "pi_usrs.hpp_ps_modelling_data"
s_table = "pi_usrs.hpp_ps_signal_data_scoring"

In [6]:
signal_list =  ['billed_amt_sum',
                 'clm_ln_cnt',
                 'clm_unit_billed',
                 'proc_code_unusual',
                 'pos_unsual',
                 'em_proc_ind',
                 'mod91_ind',
                 'modTC_ind',
                 'mod59_ind',
                 'high_cst_proc_ind',
                 'emgncy_ind']

In [7]:
modelling_data = sessn.table(m_table)
scoring_data = sessn.table(s_table)

In [8]:
scoring_data = scoring_data.select(['servicing_npi', 'srk_clm_id_rj'] + signal_list)
modelling_data = modelling_data.select(['servicing_npi'] + signal_list)

#### Getting the eligible NPIs for scoring

In [9]:
clm_th =10
org_scoring_npi_cnt = scoring_data.select("servicing_npi").distinct().count()
# Generating list Good NPIs (Having more than 10 Claims in Modelling data)
npi_clm_cnt = modelling_data.groupby("servicing_npi").count()
good_npi_clm = npi_clm_cnt.filter(F.col("count")>=clm_th)
good_npi = good_npi_clm.select("servicing_npi")
# data_to_work_scoring -> Consist scoring data for scorable NPIs
data_to_work_scoring = scoring_data.join(good_npi, on = "servicing_npi", how = "inner")
scorable_npi = data_to_work_scoring.select("servicing_npi").distinct()
scorable_npi_cnt = scorable_npi.count()
# data_to_work_modelling -> Consist modelling data for scorable NPIs
data_to_work_modelling = modelling_data.join(scorable_npi, on ="servicing_npi", how ="inner")
print("{} NPI dropped out of {}, {} NPI will be score".format(org_scoring_npi_cnt -scorable_npi_cnt, org_scoring_npi_cnt, scorable_npi_cnt))

257 NPI dropped out of 2028, 1771 NPI will be score


In [10]:
st = time.time()

#### Generating Distribution

In [11]:
scnt = len(signal_list)

In [12]:
signal_comb = list(it.product(signal_list, repeat =2 ))

In [13]:
len(signal_comb)

121

In [14]:
covarince_op = data_to_work_modelling.groupby("servicing_npi").agg(*[F.covar_samp(f[0],f[1]) for f in signal_comb])
mean_op = data_to_work_modelling.groupBy("servicing_npi").agg(*[F.mean(f) for f in signal_list])
covarince_op_pd = covarince_op.toPandas()
mean_op_pd = mean_op.toPandas()

##### Sanity checks for Covariance and Mean

In [15]:
covarince_op_pd = covarince_op_pd.round(3)

In [16]:
modelling_data_pd = data_to_work_modelling.toPandas()

In [17]:
npi_list = modelling_data_pd['servicing_npi'].unique()

In [18]:
test_op =[]
for npi in npi_list:
    sprk_op = covarince_op_pd[covarince_op_pd['servicing_npi'] == npi].iloc[:,1:].values.reshape(scnt, scnt)
    npi_data = modelling_data_pd[modelling_data_pd["servicing_npi"] == npi]
    py_op = npi_data[signal_list].cov().round(3).values
    result = (npi, (sprk_op == py_op).all())
    test_op.append(result)

In [19]:
test_op_df = pd.DataFrame(test_op)
failed_npi = test_op_df[test_op_df[1]==False][0]
test_op_df[test_op_df[1]==False]

0      1
138   1114482486  False
913   1659481141  False
1225  1326049321  False
1279  1437467776  False
1371  1326113564  False
1428  1700034956  False

In [20]:
for npi in failed_npi:
    sprk_op = covarince_op_pd[covarince_op_pd['servicing_npi'] == npi].iloc[:,1:].values.reshape(scnt, scnt)
    npi_data = modelling_data_pd[modelling_data_pd["servicing_npi"] == npi]
    py_op = npi_data[signal_list].cov().round(3).values
    matcher = (py_op == sprk_op)
    flse_index = np.where(matcher == False)
    print("{} element value are not same for {}".format(len(flse_index), npi))
    for e in flse_index:
        if len(e) == 2:
            e = tuple(e)
            print("python op {}, spark op {}".format(py_op[e], sprk_op[e]))

2 element value are not same for 1114482486
python op -0.137, spark op -0.136
python op -0.137, spark op -0.136
2 element value are not same for 1659481141
python op 1.938, spark op 1.937
python op 1.938, spark op 1.937
2 element value are not same for 1326049321
2 element value are not same for 1437467776
python op -0.158, spark op -0.157
python op -0.158, spark op -0.157
2 element value are not same for 1326113564
2 element value are not same for 1700034956


In [21]:
py_op_pd= modelling_data_pd.groupby("servicing_npi").apply(lambda x : x[signal_list].mean()).reset_index()

In [22]:
py_op = py_op_pd.sort_values(by = "servicing_npi").round(3)
sprk_op = mean_op_pd.sort_values(by = "servicing_npi").round(3)
py_op = py_op.iloc[:,1:].values
sprk_op = sprk_op.iloc[:, 1:].values
matcher = (py_op == sprk_op).any()
print(matcher)

True


#### Inverse of Covariance Matrix

In [23]:
covar_dump = covarince_op_pd.values[:,1:]
covar_dump = covar_dump.reshape(scorable_npi_cnt, scnt, scnt)
covar_npi = covarince_op_pd.values[:,0]
col_list = list(covarince_op_pd.head(1).iloc[:,1:])

In [24]:
final_list = []
np.random.seed(20)
randarr = 0.001*np.random.rand(scnt,scnt)
for npi in range(scorable_npi_cnt):
    # Adding noise to the covarinace matrix in order to avoid singular matrix 
    temp_icov = np.linalg.inv((covar_dump[npi] + randarr).astype(float))
    temp_icov = temp_icov.flatten().tolist() 
    final_list.append(temp_icov)

In [25]:
icovariance_op_pd = pd.DataFrame(final_list, columns = col_list)
icovariance_op_pd['servicing_npi'] = covarince_op_pd['servicing_npi']
icovariance_op_pd = icovariance_op_pd[list(covarince_op_pd)]

In [26]:
icovariance_op = sessn.createDataFrame(icovariance_op_pd)

### Mahalanobis in Action

<img src="http://apsrp02718.uhc.com:2222/files/jjj.jfif" style="width: 600px;"></img>

#### Right Part {u-v}*IV

##### u-v

In [27]:
ms_sprk = data_to_work_scoring.join(mean_op, how ="inner", on ="servicing_npi")

In [28]:
ms_sprk.count()

168948

In [29]:
for s in signal_list:
    s_m = s + "_m"
    s_avg = "avg({})".format(s)
    ms_sprk = ms_sprk.withColumn(s_m, F.col(s) - F.col(s_avg))
    print(s, " -> ",s_avg)
    ms_sprk = ms_sprk.drop(s)
    ms_sprk = ms_sprk.drop(s_avg)

billed_amt_sum  ->  avg(billed_amt_sum)
clm_ln_cnt  ->  avg(clm_ln_cnt)
clm_unit_billed  ->  avg(clm_unit_billed)
proc_code_unusual  ->  avg(proc_code_unusual)
pos_unsual  ->  avg(pos_unsual)
em_proc_ind  ->  avg(em_proc_ind)
mod91_ind  ->  avg(mod91_ind)
modTC_ind  ->  avg(modTC_ind)
mod59_ind  ->  avg(mod59_ind)
high_cst_proc_ind  ->  avg(high_cst_proc_ind)
emgncy_ind  ->  avg(emgncy_ind)


In [30]:
pd.DataFrame(ms_sprk.dtypes)

0       1
0         servicing_npi  string
1         srk_clm_id_rj  string
2      billed_amt_sum_m  double
3          clm_ln_cnt_m  double
4     clm_unit_billed_m  double
5   proc_code_unusual_m  double
6          pos_unsual_m  double
7         em_proc_ind_m  double
8           mod91_ind_m  double
9           modTC_ind_m  double
10          mod59_ind_m  double
11  high_cst_proc_ind_m  double
12         emgncy_ind_m  double

In [31]:
ms_sprk = ms_sprk.join(icovariance_op, how ="inner", on ="servicing_npi")

In [32]:
for s in signal_list:
    mul_list_d = []
    s_lp = s + "_lp"
    print("Working on {}".format(s_lp))
    for s2 in signal_list:
        s_m = s2 + "_m"
        s_covar = "covar_samp({}, {})".format(s,s2)
        s_m_covar_mul = s2 + "_m_covar_mul"
        ms_sprk = ms_sprk.withColumn(s_m_covar_mul, F.col(s_m)*F.col(s_covar))
        mul_list_d.append(s_m_covar_mul)
        ms_sprk = ms_sprk.drop(s_covar)
    mul_list = [F.col(x) for x in mul_list_d]
    ms_sprk = ms_sprk.withColumn(s_lp, mul_list[0] + mul_list[1] + mul_list[2] + mul_list[3] 
                             + mul_list[4] + mul_list[5] + mul_list[6] + mul_list[7] + mul_list[8]
                             + mul_list[9] + mul_list[10])
    for col_ in mul_list_d:
        ms_sprk = ms_sprk.drop(col_)

Working on billed_amt_sum_lp
Working on clm_ln_cnt_lp
Working on clm_unit_billed_lp
Working on proc_code_unusual_lp
Working on pos_unsual_lp
Working on em_proc_ind_lp
Working on mod91_ind_lp
Working on modTC_ind_lp
Working on mod59_ind_lp
Working on high_cst_proc_ind_lp
Working on emgncy_ind_lp


##### Left Part {Right Part * u_vT}

In [33]:
mul_list_d = []
for s in signal_list:
    s_lp = s + "_lp"
    s_m = s + "_m"
    s_lp_m_mul = s + "_lp_m_mul"
    ms_sprk = ms_sprk.withColumn(s_lp_m_mul, F.col(s_lp)*F.col(s_m))
    ms_sprk = ms_sprk.drop(s_lp)
    ms_sprk = ms_sprk.drop(s_m)
    mul_list_d.append(s_lp_m_mul)
mul_list = [F.col(x) for x in mul_list_d]
ms_sprk = ms_sprk.withColumn("ms",  mul_list[0] + mul_list[1] + mul_list[2] + mul_list[3] 
                             + mul_list[4] + mul_list[5] + mul_list[6] + mul_list[7] + mul_list[8]
                             + mul_list[9] + mul_list[10])
for col_ in mul_list_d:
    ms_sprk = ms_sprk.drop(col_)

#### Unit Testing 

In [34]:
srk_res_pd = ms_sprk.toPandas()

In [35]:
ms_sprk_pd = data_to_work_scoring.join(mean_op, how ="inner", on ="servicing_npi")
ms_sprk_pd = ms_sprk_pd.join(icovariance_op, how ="inner", on ="servicing_npi")

In [36]:
ms_sprk_pd = ms_sprk_pd.toPandas()

In [53]:
batch_ = np.array_split(ms_sprk_pd,100)

In [55]:
test_result_list =[]
for b in range(100):
    ms_sprk_pd_b = batch_[b]
    for sid in ms_sprk_pd_b['srk_clm_id_rj']:
        tst_data = ms_sprk_pd_b[ms_sprk_pd_b['srk_clm_id_rj'] == sid]
        npi = tst_data['servicing_npi'].values[0]
        u = tst_data[signal_list].values
        v = tst_data[mean_op.columns[1:]].values
        IV = tst_data[icovariance_op.columns[1:]].values.reshape(scnt, scnt)
        tmd = get_md(u, v, IV)
        test_result_list.append((npi, sid, tmd))

In [60]:
test_result_df = pd.DataFrame(test_result_list, columns = ["servicing_npi", "srk_clm_id_rj", "ms_python"])
test_result_df['ms_python'] = test_result_df['ms_python'].apply(lambda x : x[0][0])
test_result = srk_res_pd.merge(test_result_df, on = ["servicing_npi", "srk_clm_id_rj"], how ="inner")

In [62]:
test_result['ms'] = test_result['ms'].round(2)
test_result['ms_python'] = test_result['ms_python'].round(2)

In [63]:
test_result['status'] = test_result['ms']==test_result['ms_python']
test_failed = test_result[~test_result['status']]

In [64]:
fc = test_failed.shape[0]
tc = test_result.shape[0]
print("{} cases failed out of {}".format(fc,tc))

0 cases failed out of 168948
